# Backtest

In [ ]:
from system.agent import Agent
from system.env import Env

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }

env = Env(**config)

params_1 = {"symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM"}

In [ ]:
agent_1 = Agent(**params_1)
agent_1.update_policy_params(params = 3)
agent_1.run_episode()

In [ ]:
agent_1.get_report()

In [ ]:
env = Env(**config)
params_2 = {"symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA"}

agent_2 = Agent(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))
agent_2.run_episode()

In [ ]:
agent_2.get_report()

In [ ]:
env.journal.portfolio_data

# Interaction multi agent

In [1]:
from magent import MAgentThread
from system.env import Env


import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


config = {"capital" : 100, "interval" : "1d",
          "symbols" : ["BTC", "ETH", "SOL", "DASH", "MATIC",
                       "TWT", "GALA", "EGLD", "XMR"],
          "start" : "2023", "end" : "2023"
          }

env = Env(**config)

In [ ]:
from threading import Thread, Lock, Condition, Event, Manager

condition = Condition()

agent_msg = {}
master_msg = {}

params_1 = {"symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name" : "MOMENTUM", "condition" : condition}
params_2 = {"symbol" : "BTC", "allocation" : 100, "env" : env, "policy_name":"TRIPLEMA", "condition" : condition}


agent_1 = MAgentThread(**params_1)
agent_1.update_policy_params(params = 3)

agent_2 = MAgentThread(**params_2)
agent_2.update_policy_params(params = (7, 14, 21))

: 

In [4]:
p = {"d" : 0}
p.get("p") is None

True

In [3]:
agent_1.start()
agent_2.start()

agent_1.join()
agent_2.join()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\cc\miniconda3\envs\WcedSyst\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\cc\miniconda3\envs\WcedSyst\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\cc\Desktop\CedAlgo\C-Syst\magent.py", line 17, in run
    self.communication_lock.notify()
  File "c:\Users\cc\miniconda3\envs\WcedSyst\Lib\threading.py", line 370, in notify
    raise RuntimeError("cannot notify on un-acquired lock")
RuntimeError: cannot notify on un-acquired lock
    self.run()
  File "c:\Users\cc\Desktop\CedAlgo\C-Syst\magent.py", line 17, in run
    self.communication_lock.notify()
  File "c:\Users\cc\miniconda3\envs\WcedSyst\Lib\threading.py", line 370, in notify
    raise RuntimeError("cannot notify on un-acquired lock")
RuntimeError: cannot notify on un-acquired lock
